In [1]:
import os
import pandas as pd
import numpy as np

SRC_DIR = os.path.abspath(".")         # endereço do script
BASE_DIR = os.path.dirname(SRC_DIR)
DATA_DIR = os.path.join(BASE_DIR, "data")
IMGS_DIR = os.path.join(BASE_DIR, "images")

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import category_encoders as ce
import warnings
warnings.filterwarnings("ignore")
from sklearn import set_config
set_config(display = 'diagram')

pd.options.display.max_columns = 200
pd.options.display.max_rows = 999
pd.set_option("precision", 3)


def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution= 300):
    path=os.path.join(IMGS_DIR, fig_id + "." + fig_extension)
    print("Salving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.save_fig(path, format=fig_extension, dpi=resolution)

In [2]:
df=pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050,NaN,S


In [3]:
# retirando colunas com nome, ingresso e cabine dos conjuntos
df.drop(["Name", "Ticket", "Cabin", "PassengerId"], axis=1, inplace=True)

In [4]:
df.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
148,0,2,male,36.5,0,2,26.000,S
295,0,1,male,NaN,0,0,27.721,C
678,0,3,female,43.0,1,6,46.900,S
706,1,2,female,45.0,0,0,13.500,S
517,0,3,male,NaN,0,0,24.150,Q
839,1,1,male,NaN,0,0,29.700,C
777,1,3,female,5.0,0,0,12.475,S
468,0,3,male,NaN,0,0,7.725,Q
449,1,1,male,52.0,0,0,30.500,S
698,0,1,male,49.0,1,1,110.883,C


In [5]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000,891.000,714.000,891.000,891.000,891.000
mean,0.384,2.309,29.699,0.523,0.382,32.204
std,0.487,0.836,14.526,1.103,0.806,49.693
min,0.000,1.000,0.420,0.000,0.000,0.000
25%,0.000,2.000,20.125,0.000,0.000,7.910
50%,0.000,3.000,28.000,0.000,0.000,14.454
75%,1.000,3.000,38.000,1.000,0.000,31.000
max,1.000,3.000,80.000,8.000,6.000,512.329


In [6]:
df.describe(include=['O'])

,Sex,Embarked
count,891,889
unique,2,3
top,male,S
freq,577,644


Preencher a linha a amostra faltante da variável "Embarked"

In [7]:
df.Embarked.mode()

0    S
dtype: object

In [8]:
df.fillna({'Embarked':'S'}, inplace=True)
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [9]:
df['family_size'] = df.SibSp + df.Parch + 1

In [10]:
df_age=df[~df['Age'].isna()].copy() # pegando todas as linhas aonde Age não é NaN
df_age

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size
0,0,3,male,22.00,1,0,7.250,S,2
1,1,1,female,38.00,1,0,71.283,C,2
2,1,3,female,26.00,0,0,7.925,S,1
3,1,1,female,35.00,1,0,53.100,S,2
4,0,3,male,35.00,0,0,8.050,S,1
6,0,1,male,54.00,0,0,51.862,S,1
7,0,3,male,2.00,3,1,21.075,S,5
8,1,3,female,27.00,0,2,11.133,S,3
9,1,2,female,14.00,1,0,30.071,C,2
10,1,3,female,4.00,1,1,16.700,S,3


Usando decision tree regressor para fazer a inputação da idade nas linhas aonde a idade está faltando.

In [11]:
from sklearn.tree import DecisionTreeRegressor
features_age = df_age.select_dtypes(include=['int', 'float']).drop('Age', axis=1).columns
features_age

Index(['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare', 'family_size'], dtype='object')

In [12]:
tree_reg_age=DecisionTreeRegressor(max_depth=3, random_state=123)  # instanciando um decision tree regressor para calcular as idades faltantes
tree_reg_age.fit(df_age[features_age], df_age['Age'])

DecisionTreeRegressor(max_depth=3, random_state=123)

In [13]:
df['AgePredicted'] = tree_reg_age.predict(df[features_age])
df['AgePredicted'].head()

0    27.835
1    37.726
2    27.835
3    37.726
4    27.835
Name: AgePredicted, dtype: float64

In [14]:
def input_age(row):
    if np.isnan(row['Age']):
        return row['AgePredicted']
    else:
        return row['Age']


df["Age"]= df[["Age", "AgePredicted"]].apply(input_age, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Survived      891 non-null    int64  
 1   Pclass        891 non-null    int64  
 2   Sex           891 non-null    object 
 3   Age           891 non-null    float64
 4   SibSp         891 non-null    int64  
 5   Parch         891 non-null    int64  
 6   Fare          891 non-null    float64
 7   Embarked      891 non-null    object 
 8   family_size   891 non-null    int64  
 9   AgePredicted  891 non-null    float64
dtypes: float64(3), int64(5), object(2)
memory usage: 69.7+ KB


In [15]:
df.drop('AgePredicted', axis=1, inplace=True)
df.dtypes

Survived         int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
family_size      int64
dtype: object

In [16]:
df['Pclass']=(df['Pclass'].map({1: 'first', 2: 'second', 3:'third'})
                          .astype('category'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Survived     891 non-null    int64   
 1   Pclass       891 non-null    category
 2   Sex          891 non-null    object  
 3   Age          891 non-null    float64 
 4   SibSp        891 non-null    int64   
 5   Parch        891 non-null    int64   
 6   Fare         891 non-null    float64 
 7   Embarked     891 non-null    object  
 8   family_size  891 non-null    int64   
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 56.8+ KB


In [17]:
numeric_features=df.select_dtypes(include=['int64', 'float64']).drop('Survived', axis=1).columns
categoric_features=df.select_dtypes(include=['object', 'category']).columns

In [18]:
numeric_features

Index(['Age', 'SibSp', 'Parch', 'Fare', 'family_size'], dtype='object')

In [19]:
categoric_features

Index(['Pclass', 'Sex', 'Embarked'], dtype='object')

In [20]:
nominal_features = ['Sex', 'Embarked']
ordinal_features = ['Pclass']

In [21]:
features=df.drop('Survived', axis=1).copy()
label = df[['Survived']].copy()
display(features.head())
display(label.head())

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size
0,third,male,22.0,1,0,7.250,S,2
1,first,female,38.0,1,0,71.283,C,2
2,third,female,26.0,0,0,7.925,S,1
3,first,female,35.0,1,0,53.100,S,2
4,third,male,35.0,0,0,8.050,S,1


,Survived
0,0
1,1
2,1
3,1
4,0


In [22]:
# dividindo em conjunto de treino e test
X = features
y = label

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2, 
                                                    random_state=42)

In [23]:
display(X_train.shape, X_test.shape)

(712, 8)

(179, 8)

In [24]:
display(y_train.shape, y_test.shape)

(712, 1)

(179, 1)

## Criando o pipeline de preprocessamento
***

In [25]:
# Compondo os preprocessadores
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categoric_transformer_1 = Pipeline(steps=[
    ('onehot', ce.OneHotEncoder())
])

categoric_transformer_2 = Pipeline(steps=[
    ('ordinal', ce.OrdinalEncoder())])

preprocessor = ColumnTransformer(transformers=[
('num', numeric_transformer, numeric_features),
('cat_nominal', categoric_transformer_1, nominal_features),
('cat_ordinal', categoric_transformer_2, ordinal_features)
])

In [26]:
tree = Pipeline(steps=[('preprocessor', preprocessor),
       ('clf', DecisionTreeClassifier(random_state=123))
])

In [27]:
tree.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'SibSp', 'Parch', 'Fare', 'family_size'], dtype='object')),
                                                 ('cat_nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked']),
                                                 ('cat_ordinal',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Pclass'])])),
                ('clf', DecisionTreeClassifier(random_state=123))])

In [28]:
tree.score(X_test, y_test)

0.8044692737430168

# Avaliando modelos de classificação
***

1.Acurácia: A porcentagem(ou proporção de casos classificados corretamente)

2.Confusion matrix: Uma tabela 2x2 - para problemas binários, formado pela contagem do status da classificação predita e a atual.

3.Sensibilidade: A porcentagem ou proporção valores 1s corretamente classificados.Recall

4.ROC curve: Um plot de sensitividade x especitividade

In [29]:
from sklearn.metrics import(accuracy_score,
                            precision_score,
                            recall_score,
                            confusion_matrix)
y_pred=tree.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)
prec = precision_score(y_test, y_pred)
print(prec)
recall = recall_score(y_test, y_pred)
print(recall)

0.8044692737430168
0.7671232876712328
0.7567567567567568


In [30]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [35]:
tree.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'clf', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__num', 'preprocessor__cat_nominal', 'preprocessor__cat_ordinal', 'preprocessor__num__memory', 'preprocessor__num__steps', 'preprocessor__num__verbose', 'preprocessor__num__scaler', 'preprocessor__num__scaler__copy', 'preprocessor__num__scaler__with_mean', 'preprocessor__num__scaler__with_std', 'preprocessor__cat_nominal__memory', 'preprocessor__cat_nominal__steps', 'preprocessor__cat_nominal__verbose', 'preprocessor__cat_nominal__onehot', 'preprocessor__cat_nominal__onehot__cols', 'preprocessor__cat_nominal__onehot__drop_invariant', 'preprocessor__cat_nominal__onehot__handle_missing', 'preprocessor__cat_nominal__onehot__handle_unknown', 'preprocessor__cat_nominal__onehot__return_df', 'preprocessor__cat_nominal__onehot__use_cat_names', 

In [36]:
# Tunando hiperparâmetros com 5-fold cross-validation e pipelines
parameters = {'clf__max_depth': [3, 4, 5, 6, 7],
              'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(tree, param_grid=parameters, cv=kfold, n_jobs=-1, return_train_score=True)
grid.fit(X=df.drop(['Survived'], axis=1), y=df['Survived'])


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'SibSp', 'Parch', 'Fare', 'family_size'], dtype='object')),
                                                                        ('cat_nominal',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder())]),
                                                                         ['Sex',
                                                                          'Embarked']),
                                                                        ('cat_ordinal',
                                                                         Pipeline(steps=[('ordinal',
                                                                                          OrdinalEncoder())]),
                                                                         ['Pclass'])])),
                                       ('clf',
                                        DecisionTreeClassifier(random_state=123))]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [3, 4, 5, 6, 7],
                         'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
             return_train_score=True)

In [37]:
y_pred=tree.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)
prec = precision_score(y_test, y_pred)
print(prec)
recall = recall_score(y_test, y_pred)
print(recall)

0.8044692737430168
0.7671232876712328
0.7567567567567568


In [38]:
from sklearn.linear_model import LogisticRegression

logistic_model = Pipeline(steps=[('preprocessor', preprocessor),
       ('clf_logistic', LogisticRegression(penalty='l2',
                                           C=1.0, 
                                           solver='liblinear'))
])
logistic_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'SibSp', 'Parch', 'Fare', 'family_size'], dtype='object')),
                                                 ('cat_nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked']),
                                                 ('cat_ordinal',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Pclass'])])),
                ('clf_logistic', LogisticRegression(solver='liblinear'))])

In [39]:
y_pred = logistic_model.predict(X_test)
pred_results = pd.DataFrame(
    {'y_test':y_test,
     'y_pred':y_pred}
)

ValueError: Shape of passed values is (1, 2), indices imply (179, 2)